This example notebook closely follows the [Databricks documentation](https://docs.azuredatabricks.net/spark/latest/data-sources/azure/azure-datalake.html) for how to set up Azure Data Lake Store as a data source in Databricks.

### 0 - Setup

To get set up, do these tasks first: 

- Get service credentials: Client ID `<aaaaaaaa-bbbb-cccc-dddd-eeeeeeeeeeee>` and Client Credential `<NzQzY2QzYTAtM2I3Zi00NzFmLWI3MGMtMzc4MzRjZmk=>`. Follow the instructions in [Create service principal with portal](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-create-service-principal-portal). 
- Get directory ID `<ffffffff-gggg-hhhh-iiii-jjjjjjjjjjjj>`: This is also referred to as *tenant ID*. Follow the instructions in [Get tenant ID](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-create-service-principal-portal#get-tenant-id). 
- If you haven't set up the service app, follow this [tutorial](https://docs.microsoft.com/en-us/azure/azure-databricks/databricks-extract-load-sql-data-warehouse). Set access at the root directory or desired folder level to the service or everyone.

There are two options to read and write Azure Data Lake data from Azure Databricks:
1. DBFS mount points
2. Spark configs

## 1 - DBFS mount points
[DBFS](https://docs.azuredatabricks.net/user-guide/dbfs-databricks-file-system.html) mount points let you mount Azure Data Lake Store for all users in the workspace. Once it is mounted, the data can be accessed directly via a DBFS path from all clusters, without the need for providing credentials every time. The example below shows how to set up a mount point for Azure Data Lake Store.

In [0]:
%scala
val configs = Map(
  "dfs.adls.oauth2.access.token.provider.type" -> "ClientCredential",
  "dfs.adls.oauth2.client.id" -> "<aaaaaaaa-bbbb-cccc-dddd-eeeeeeeeeeee>",
  "dfs.adls.oauth2.credential" -> "<NzQzY2QzYTAtM2I3Zi00NzFmLWI3MGMtMzc4MzRjZmk=>",
  "dfs.adls.oauth2.refresh.url" -> "https://login.microsoftonline.com/<ffffffff-gggg-hhhh-iiii-jjjjjjjjjjjj>/oauth2/token")

dbutils.fs.mount(
  source = "adl://kpadls.azuredatalakestore.net/",
  mountPoint = "/mnt/kp-adls",
  extraConfigs = configs)

In [0]:
%fs ls /mnt/kp-adls-testing

##2 - Spark Configs

With Spark configs, the Azure Data Lake Store settings can be specified per notebook. To keep things simple, the example below includes the credentials in plaintext. However, we strongly discourage you from storing secrets in plaintext. Instead, we recommend storing the credentials as [Databricks Secrets](https://docs.azuredatabricks.net/user-guide/secrets/index.html#secrets-user-guide).

**Note:** `spark.conf` values are visible only to the DataSet and DataFrames API. If you need access to them from an RDD, refer to the [documentation](https://docs.azuredatabricks.net/spark/latest/data-sources/azure/azure-datalake.html#access-azure-data-lake-store-using-the-rdd-api).

In [0]:
%scala
spark.conf.set("dfs.adls.oauth2.access.token.provider.type", "ClientCredential")
spark.conf.set("dfs.adls.oauth2.client.id", "<aaaaaaaa-bbbb-cccc-dddd-eeeeeeeeeeee>")
spark.conf.set("dfs.adls.oauth2.credential", "<NzQzY2QzYTAtM2I3Zi00NzFmLWI3MGMtMzc4MzRjZmk=>")
spark.conf.set("dfs.adls.oauth2.refresh.url", "https://login.microsoftonline.com/<ffffffff-gggg-hhhh-iiii-jjjjjjjjjjjj>/oauth2/token")

In [0]:
%fs ls adl://kpadls.azuredatalakestore.net/testing/

In [0]:
%scala
spark.read.parquet("dbfs:/mnt/my-datasets/datasets/iot/events").write.mode("overwrite").parquet("adl://kpadls.azuredatalakestore.net/testing/tmp/kp/v1")

In [0]:
%fs ls adl://kpadls.azuredatalakestore.net/testing/tmp/kp/v1

In [0]:
dbutils.fs.mount(
source = "wasbs://bronze1@grpstorageacc.blob.core.windows.net",
mount_point='/mnt/sb1',
extra_configs={
'fs.azure.account.key.grpstorageacc.blob.core.windows.net':'VLDH5xr3RnZoy9vNw7KYf1We16brXes2sVjRvsEiG+90JplEmRNnXVcB7hlEmIc77DOimqsZaj7Z+AStNIPHRg=='
}
)

Out[144]: True

In [0]:
dbutils.fs.ls('/mnt/sb1')

Out[145]: [FileInfo(path='dbfs:/mnt/sb1/BIKE DETAILS.csv', name='BIKE DETAILS.csv', size=66450, modificationTime=1667383703000),
 FileInfo(path='dbfs:/mnt/sb1/bike_details.csv', name='bike_details.csv', size=66450, modificationTime=1667400025000)]

In [0]:
import pandas as pd

In [0]:
df_bdts=pd.read_csv('/dbfs/mnt/sb1/bike_details.csv')

In [0]:
df_bdts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061 entries, 0 to 1060
Data columns (total 7 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 name 1061 non-null object 
 1 selling_price 1061 non-null int64 
 2 year 1061 non-null int64 
 3 seller_type 1061 non-null object 
 4 owner 1061 non-null object 
 5 km_driven 1061 non-null int64 
 6 ex_showroom_price 626 non-null float64
dtypes: float64(1), int64(3), object(3)
memory usage: 58.1+ KB

In [0]:
df_bdts.head()

Out[170]:

,name,selling_price,year,seller_type,owner,km_driven,ex_showroom_price
0,Royal Enfield Classic 350,175000,2019,Individual,1st owner,350,NaN
1,Honda Dio,45000,2017,Individual,1st owner,5650,NaN
2,Royal Enfield Classic Gunmetal Grey,150000,2018,Individual,1st owner,12000,148114.0
3,Yamaha Fazer FI V 2.0 [2016-2018],65000,2015,Individual,1st owner,23000,89643.0
4,Yamaha SZ [2013-2014],20000,2011,Individual,2nd owner,21000,NaN


In [0]:
df_bdts.drop_duplicates()

Out[171]:

,name,selling_price,year,seller_type,owner,km_driven,ex_showroom_price
0,Royal Enfield Classic 350,175000,2019,Individual,1st owner,350,NaN
1,Honda Dio,45000,2017,Individual,1st owner,5650,NaN
2,Royal Enfield Classic Gunmetal Grey,150000,2018,Individual,1st owner,12000,148114.0
3,Yamaha Fazer FI V 2.0 [2016-2018],65000,2015,Individual,1st owner,23000,89643.0
4,Yamaha SZ [2013-2014],20000,2011,Individual,2nd owner,21000,NaN
...,...,...,...,...,...,...,...
1056,Activa 3g,17000,2010,Individual,1st owner,500000,52000.0
1057,Honda CB twister,16000,2012,Individual,1st owner,33000,51000.0
1058,Bajaj Discover 125,15000,2013,Individual,2nd owner,35000,57000.0
1059,Honda CB Shine,12000,2009,Individual,1st owner,53000,58000.0


In [0]:
df_bd=df_bd['selling_price'].min()
df_bd

Out[163]: 5000

In [0]:
df_bdts['ex_showroom_price']=df_bdts['ex_showroom_price'].fillna(4999.0)

In [0]:
df_bdts.head()

Out[173]:

,name,selling_price,year,seller_type,owner,km_driven,ex_showroom_price
0,Royal Enfield Classic 350,175000,2019,Individual,1st owner,350,4999.0
1,Honda Dio,45000,2017,Individual,1st owner,5650,4999.0
2,Royal Enfield Classic Gunmetal Grey,150000,2018,Individual,1st owner,12000,148114.0
3,Yamaha Fazer FI V 2.0 [2016-2018],65000,2015,Individual,1st owner,23000,89643.0
4,Yamaha SZ [2013-2014],20000,2011,Individual,2nd owner,21000,4999.0


In [0]:
dbutils.fs.mount(
source = "wasbs://silver2@grpstorageacc.blob.core.windows.net",
mount_point='/mnt/sb2',
extra_configs={
'fs.azure.account.key.grpstorageacc.blob.core.windows.net':'VLDH5xr3RnZoy9vNw7KYf1We16brXes2sVjRvsEiG+90JplEmRNnXVcB7hlEmIc77DOimqsZaj7Z+AStNIPHRg=='
}
)

Out[174]: True

In [0]:
df_bdts=spark.createDataFrame(df_bdts)

In [0]:
df_bdts.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").mode("overwrite").save("/mnt/sb2/Bikedetails.csv")

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-1258457334936385> in <module> 
 ----> 1 df_bdts . coalesce ( 1 ) . write . format ( "com.databricks.spark.csv" ) . option ( "header" , "true" ) . mode ( "overwrite" ) . save ( "/mnt/sb2/Bikedetails.csv" ) 

 /databricks/python/lib/python3.8/site-packages/pandas/core/generic.py in __getattr__ (self, name) 
 5463 if self . _info_axis . _can_hold_identifiers_and_holds_name ( name ) : 
 5464 return self [ name ] 
 -> 5465 return object . __getattribute__ ( self , name ) 
 5466 
 5467 def __setattr__ ( self , name : str , value ) -> None : 

 AttributeError : 'DataFrame' object has no attribute 'coalesce'

In [0]:
dbutils.fs.ls('/mnt/sb2/Bikedetails.csv')

Out[178]: [FileInfo(path='dbfs:/mnt/sb2/Bikedetails.csv/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1667554002000),
 FileInfo(path='dbfs:/mnt/sb2/Bikedetails.csv/_committed_6876077998823395531', name='_committed_6876077998823395531', size=111, modificationTime=1667554002000),
 FileInfo(path='dbfs:/mnt/sb2/Bikedetails.csv/_started_6876077998823395531', name='_started_6876077998823395531', size=0, modificationTime=1667554001000),
 FileInfo(path='dbfs:/mnt/sb2/Bikedetails.csv/bike', name='bike', size=69245, modificationTime=1667554001000)]

In [0]:
df_bdts=pd.read_csv('/dbfs/mnt/sb2/Bikedetails.csv/bike')

In [0]:
df_bdts=df_bdts.groupby(['year'])['selling_price'].sum().sort_values(ascending=False).head(10)
print(df_bdts)


year
2018 11483509
2017 10493000
2019 10293298
2016 6256185
2015 5650000
2014 4428800
2013 3733000
2012 2502388
2011 2174999
2010 1907600
Name: selling_price, dtype: int64

In [0]:
df_bdts.head()

Out[101]: year
2018 11483509
2017 10493000
2019 10293298
2016 6256185
2015 5650000
Name: selling_price, dtype: int64

In [0]:

df_bdts= df_bdts.assign(Profit_or_Loss=lambda x: x.selling_price - x.ex_showroom_price)
print(df_bdts)

name ... Profit_or_Loss
0 Royal Enfield Classic 350 ... 170001.0
1 Honda Dio ... 40001.0
2 Royal Enfield Classic Gunmetal Grey ... 1886.0
3 Yamaha Fazer FI V 2.0 [2016-2018] ... -24643.0
4 Yamaha SZ [2013-2014] ... 15001.0
... ... ... ...
1056 Activa 3g ... -35000.0
1057 Honda CB twister ... -35000.0
1058 Bajaj Discover 125 ... -42000.0
1059 Honda CB Shine ... -46000.0
1060 Bajaj Pulsar 150 ... -65000.0

[1061 rows x 8 columns]

In [0]:
df_bdts.head(10)

Out[183]:

,name,selling_price,year,seller_type,owner,km_driven,ex_showroom_price,Profit_or_Loss
0,Royal Enfield Classic 350,175000,2019,Individual,1st owner,350,4999.0,170001.0
1,Honda Dio,45000,2017,Individual,1st owner,5650,4999.0,40001.0
2,Royal Enfield Classic Gunmetal Grey,150000,2018,Individual,1st owner,12000,148114.0,1886.0
3,Yamaha Fazer FI V 2.0 [2016-2018],65000,2015,Individual,1st owner,23000,89643.0,-24643.0
4,Yamaha SZ [2013-2014],20000,2011,Individual,2nd owner,21000,4999.0,15001.0
5,Honda CB Twister,18000,2010,Individual,1st owner,60000,53857.0,-35857.0
6,Honda CB Hornet 160R,78500,2018,Individual,1st owner,17000,87719.0,-9219.0
7,Royal Enfield Bullet 350 [2007-2011],180000,2008,Individual,2nd owner,39000,4999.0,175001.0
8,Hero Honda CBZ extreme,30000,2010,Individual,1st owner,32000,4999.0,25001.0
9,Bajaj Discover 125,50000,2016,Individual,1st owner,42000,60122.0,-10122.0


In [0]:
len(df_bdts)

Out[104]: 1061

In [0]:
df_bdts.describe()

Out[184]:

,selling_price,year,km_driven,ex_showroom_price,Profit_or_Loss
count,1061.000000,1061.000000,1061.000000,1.061000e+03,1061.000000
mean,59638.151744,2013.867107,34359.833176,5.394601e+04,5692.138549
std,56304.291973,4.301191,51623.152702,7.216301e+04,58032.676482
min,5000.000000,1988.000000,350.000000,4.999000e+03,-528000.000000
25%,28000.000000,2011.000000,13500.000000,4.999000e+03,-27545.000000
50%,45000.000000,2015.000000,25000.000000,5.227100e+04,-8147.000000
75%,70000.000000,2017.000000,43000.000000,7.851300e+04,30001.000000
max,760000.000000,2020.000000,880000.000000,1.278000e+06,294001.000000


In [0]:
df_bdts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061 entries, 0 to 1060
Data columns (total 8 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 name 1061 non-null object 
 1 selling_price 1061 non-null int64 
 2 year 1061 non-null int64 
 3 seller_type 1061 non-null object 
 4 owner 1061 non-null object 
 5 km_driven 1061 non-null int64 
 6 ex_showroom_price 1061 non-null float64
 7 Profit_or_Loss 1061 non-null float64
dtypes: float64(2), int64(3), object(3)
memory usage: 66.4+ KB

In [0]:
df_bdts.isnull().sum()

Out[186]: name 0
selling_price 0
year 0
seller_type 0
owner 0
km_driven 0
ex_showroom_price 0
Profit_or_Loss 0
dtype: int64

In [0]:
df_bdts.notnull().sum()

Out[187]: name 1061
selling_price 1061
year 1061
seller_type 1061
owner 1061
km_driven 1061
ex_showroom_price 1061
Profit_or_Loss 1061
dtype: int64

In [0]:

df_bdts=df_bdts.groupby(['year'])['selling_price'].sum()
df_bdts.head(10)

Out[109]: year
1988 20000
1991 6000
1993 130000
1995 150000
1997 35000
1998 85000
1999 15000
2000 125000
2001 80000
2002 62000
Name: selling_price, dtype: int64

In [0]:
df_bdts=df_bdts.groupby(['year'])['selling_price'].count()
df_bdts.head(30)

Out[110]: year
1988 1
1991 1
1993 1
1995 1
1997 2
1998 3
1999 1
2000 6
2001 2
2002 3
2003 1
2004 5
2005 14
2006 20
2007 29
2008 28
2009 28
2010 60
2011 61
2012 70
2013 73
2014 91
2015 100
2016 107
2017 133
2018 131
2019 86
2020 3
Name: selling_price, dtype: int64

In [0]:
df_bdts['Profit_or_Loss'] = df_bdts['Profit_or_Loss'].apply(pd.to_numeric)

In [0]:
df_bdts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061 entries, 0 to 1060
Data columns (total 8 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 name 1061 non-null object 
 1 selling_price 1061 non-null int64 
 2 year 1061 non-null int64 
 3 seller_type 1061 non-null object 
 4 owner 1061 non-null object 
 5 km_driven 1061 non-null int64 
 6 ex_showroom_price 1061 non-null float64
 7 Profit_or_Loss 1061 non-null float64
dtypes: float64(2), int64(3), object(3)
memory usage: 66.4+ KB

In [0]:
dbutils.fs.mount(
source = "wasbs://gold3@grpstorageacc.blob.core.windows.net",
mount_point='/mnt/sb3',
extra_configs={
'fs.azure.account.key.grpstorageacc.blob.core.windows.net':'VLDH5xr3RnZoy9vNw7KYf1We16brXes2sVjRvsEiG+90JplEmRNnXVcB7hlEmIc77DOimqsZaj7Z+AStNIPHRg=='
}
)

Out[190]: True

In [0]:
df_bdts=spark.createDataFrame(df_bdts)

In [0]:
df_bdts.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").mode("overwrite").save("/mnt/sb3/bdetails.csv")